In [3]:
import sys # biblioteca para controlar a execução do programa
import vlc # Importa o VLC media player, necessário para a execução do video
import os # Módulo de manipulação do sistema operacional, permitindo a interação com o OS

# PyQt5.QtWidgets contém os elemenos da interface gráfica, como botões, janelas e sliders
# QApplication - Responsável por gerenciar toda a aplicação PyQt
# QMainWindow - Representa a janela principal de um aplicativo PyQt. Permite adicionar menus, barras de ferramentas e widgets centrais.
# QPushButton - Cria um botão interativo. Pode ser clicado para executar ações quando conectado a um slot (função).
# QFileDialog - Exibe uma caixa de diálogo para abrir ou salvar arquivos.
# QSlider - Cria uma barra deslizante para selecionar valores numéricos
# QWidget - Classe base de qualquer componente de interface. Pode ser usado como um container genérico.
# QVBoxLayout - Gerencia o layout na vertical (empilha os widgets um abaixo do outro).
# QHBoxLayout - Gerencia o layout na horizontal (coloca widgets lado a lado).
# QLabel - Exibe texto ou imagens estáticas.
# QShortcut - Cria atalhos de teclado para ações específicas
# QDialog - 
# QLineEdit - 
# QScrollArea -
# QGridLayout -

from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QFileDialog, QSlider, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QShortcut, QDialog, QLineEdit, QScrollArea, QGridLayout

# O módulo QtCore contém funcionalidades não gráficas, como controle de tempo, eventos e manipulação de dados.
# Qt - Enumerações e constantes do Qt, como: Qt.AlignTop → Alinha um widget no topo.
# QTimer - Cria eventos recorrentes, como atualização de tempo ou animações.

from PyQt5.QtCore import Qt, QTimer

# O módulo QtGui lida com elementos gráficos, como fontes, cores e atalhos de teclado.
# QKeySequence - Representa atalhos de teclado, como "Ctrl+S" ou "Alt+F4"
# QFont - Define e personaliza fontes de texto.
# QPixmap -

from PyQt5.QtGui import QKeySequence, QFont, QPixmap

# O módulo QtMultimediaWidgets permite manipular mídia dentro da GUI.
# QVideoWidget - Cria uma área de exibiçãi de vídeo embutido dentro da interface. 
# Ele exibe vídeos, mas não contém um player próprio.

from PyQt5.QtMultimediaWidgets import QVideoWidget

In [4]:
# Classe responsável por exibir e gerenciar a janela do Menu de saves
class SaveMenu(QDialog):

    # Construtor da classe
    def __init__(self, videoName):
        
        # Realiza a chamada dos métodos da classe pai (QDialog)
        super().__init__()

        self.setWindowTitle("Menu de salvamento") # Define o título da janela
        self.setGeometry(150, 150, 800, 600) # Define as dimensões da janela criada

        # Layout principal do QDialog
        # Em um QDialog, para que os widgets apareçam, é necessário 
        # definir um layout principal para a própria janela (self).
        self.layout = QVBoxLayout(self)

        # Define um dicionário para definir a categoria de cada pasta, bem como o caminho
        # Dicionário -> "Categoria" : "Caminho"
        # Nesse caso, os valores são iguais por que as pastas estão no mesmo diretório do arquivo .py
        # com isso, não precisa especificar todo o caminho
        self.folders = {
            "Indolor": "Indolor",
            "Pouca dor": "Pouca dor",
            "Muita dor": "Muita dor"
        }

        # Criar uma área de rolagem para exibir imagens, independente
        # do tamanho da janela
        self.scroll_area = QScrollArea(self) # Instância da área de rolagem (associada à propria instância da janela - self)
        self.scroll_widget = QWidget() # O QScrollArea sozinho não exibe widgets diretamente. Ele precisa de um widget interno para servir de "conteúdo" rolável
        
        self.scroll_layout = QVBoxLayout(self.scroll_widget) # Criamos um layout vertical (QVBoxLayout) para organizar os elementos dentro do scroll_widget
        self.scroll_area.setWidgetResizable(True) # Isso permite que o conteúdo interno (self.scroll_widget) seja redimensionado automaticamente para caber na QScrollArea
        self.scroll_area.setWidget(self.scroll_widget) # Associando o widget que contém nosso layout de imagens à área de rolagem 
        self.layout.addWidget(self.scroll_area) # Adicionar a área de rolagem ao layout principal

        # ---------------------------------------------------------------------------------------------------------------------------

        self.close_btn = QPushButton("Fechar") # Cria um botão para fechar a janela
        self.layout.addWidget(self.close_btn) # Adiciona o botão ao layout principal da janela
        self.close_btn.clicked.connect(self.close) # Atribui a função de fechar a janela ao botão

        # ---------------------------------------------------------------------------------------------------------------------------
        
        # Recebe o nome do vídeo atual
        self.videoName = videoName

        # Chamada de função responsáverl por carregar imagens das pastas
        self.load_images()

    # A função a seguir serve para quando é necessário atualizar um layout removendo os elementos antigos antes de adicionar novos.
    def clear_layout(self, layout):
        
        # layout.count() retorna o número de widgets ou sublayouts dentro do layout.
        # O while garante que os elementos continuem sendo removidos até que não reste nenhum.
        while layout.count():

            # Remove o primeiro item do layout e o retorna
            # É importante ressaltar que Remove o QLayout pode 
            # conter tanto widgets (QWidget) quanto 
            # sublayouts (QLayout), é necessário verificar o que foi removido.
            item = layout.takeAt(0)

            # Verifica se o item é um widget
            if item.widget():
                # Remove o widget da memória de forma segura. 
                # O deleteLater() é usado no PyQt para evitar 
                # problemas de acesso a objetos deletados prematuramente.
                item.widget().deleteLater()

            # Verifica se o item é layout
            elif item.layout():
                
                # A propria função é chamada recursivamente de forma a limpar todos
                # os widgets contidos nele
                self.clear_layout(item.layout())  
                item.layout().deleteLater() #Remove o layout da memória após limpá-lo.

    # Função responsável por carregar e exibir imagens das pastas especificadas na interface gráfica.
    def load_images(self):
        
        # Limpa os widgets existentes no layout antes de recarregar novas imagens.
        # Evitando que a interface acumule elementos repetidos
        self.clear_layout(self.scroll_layout)

        # Percorre cada um dos itens do dicionário das pastas (retornando a categoria e o caminho)
        # possibilitando a realização do processo de carregamento de imagens para cada uma das pastas

        for category, path in self.folders.items():
            
            # Se a pasta não existir no caminho definido em 'path'
            # o algoritmo pula a pasta e continua a execução

            if not os.path.exists(path):
                continue  

            # Adiciona um layout como título para cada categoria
            # o label recebe o nome da categoria obtido pela
            # variável 'category' proveniente do dicionário

            category_label = QLabel(f"<b>{category}</b>")
            self.scroll_layout.addWidget(category_label)

            # Criar um layout de grade para organizar as imagens
            # A grid é definida para cada uma das pastas

            grid_layout = QGridLayout()
            row, col = 0, 0 # Inicia a posição da primeira imagem na linha 0, coluna 0.


            # O loop percorre todos os itens em uma determinada pasta
            # definida por 'path'
            for img_file in os.listdir(path):
                
                # Valida quais arquivos são referentes ao vídeo atual
                if self.videoName in img_file:

                    # Obtém o caminho completo do arquivo (unindo path com o nome do arquivo de imagem)
                    img_path = os.path.join(path, img_file)

                    # Valida se o arquivo da pasta possui a terminação '.png' ou '.jpg'
                    # Em outras palavras, é validado se o arquivo é uma imagem. Caso não seja
                    # validado, ele será apenas ignorado

                    if not img_file.lower().endswith(('.png', '.jpg')):
                        continue  

                    # Criar um QLabel para exibir a imagem
                    # Para isso é utilizado um pixmap, o qual é
                    # designado para exibir imagens de modo otimizado
                    # sem ter foco na manipulação da imagem
                    pixmap = QPixmap(img_path).scaled(100, 100) # Instância do pixmap da imagem presente no caminho img_path
                    img_label = QLabel() # Instância de uma label que contém o pixmap
                    img_label.setPixmap(pixmap) # Associa o pixmap ao label da imagem

                    # Criar um botão para excluir a imagem
                    delete_button = QPushButton("Excluir")
                    
                    # Ao ser pressionado o botão chama a função delete_image
                    # Lambda permite a chamada de uma função com parâmetros 
                    
                    # Observação:
                    # O evento clicked do botão envia automaticamente um argumento booleano (checked)
                    # O checked não é usado, mas precisa estar no lambda para evitar erros, uma vez que 
                    # alguns widgets no PyQt podem funcionar como botões de alternância (toggle buttons), 
                    # enviando True quando ativados e False quando desativados. Mesmo que o nosso botão não 
                    # use isso, o PyQt exige que esse argumento seja tratado.
                    delete_button.clicked.connect(lambda checked, p = img_path: self.delete_image(p))

                    # Label para exibir o nome de cada frame
                    frame_name = QLabel()
                    frame_name.setText(f"{img_file}")

                    # Adicionar a imagem do frame e o botão de deleção
                    # ao layout da grid, definindo a linha e a coluna
                    # da grid onde será a inserção
                    grid_layout.addWidget(frame_name, row, col)
                    grid_layout.addWidget(img_label, row + 1, col)
                    grid_layout.addWidget(delete_button, row + 2, col) # A inclusão do botão vem na linha inferior de onde a imagem está alocada

                    # Aqui é feito o controle da disposição das imagens
                    # A coluna é incrementada após a inserção da primeira imagem
                    col += 1
                    # Caso já tenham 5 colunas de imagens (0,1,2,3,4)
                    # a proxima imagem é inserida na próxima linha da grid
                    # resetando o valor da coluna
                    if col > 4:  
                        col = 0
                        row += 3

            # Após a definição e organização da grid, o seu layout
            # é inserido ao scroll de rolagem
            self.scroll_layout.addLayout(grid_layout)

    # Função responsável por remover o arquivo presente no caminho
    # definido por 'img_path'
    def delete_image(self, img_path):
        
        try:
            os.remove(img_path)  # Remove o arquivo
            self.load_images()  # Recarrega a interface após a exclusão

        except Exception as e: # Em caso de erro
            print(f"Erro ao excluir {img_path}: {e}")

In [5]:
# Classe responsável pela janela de configuração da atribuição de teclas
class KeyMapper(QDialog):

    def __init__(self):
        
        super().__init__()

        self.setWindowTitle("Mapear teclas de atalho")
        self.setGeometry(100, 100, 250, 120)

        # Layout principal do QDialog
        # Em um QDialog, para que os widgets apareçam, é necessário 
        # definir um layout principal para a própria janela (self).
        self.dialog_layout = QVBoxLayout(self)

        # Criando um widget dedicado para os controles, com uma altura fixa
        self.main_widget = QWidget() # Cria uma instância do QWidget
        self.main_widget.setFixedHeight(500)  # Define uma altura fixa para os controles
        
        # Criando um layout específico para o main_widget
        self.main_layout = QVBoxLayout(self.main_widget) # Define um layout vertical para os controles
        self.main_layout.setAlignment(Qt.AlignTop) # Controles alinhados ao topo

        # Adicionando o widget de controles ao layout principal do QDialog
        self.dialog_layout.addWidget(self.main_widget)

        # ------------------------------------------------------------------------------------------------

        self.keyValues = []

        # ------------------------------------------------------------------------------------------------

        # Criando os campos de entrada com seus respectivos rótulos

        #self.input1_layout = QHBoxLayout() -> Layout horizontal para acomodar o label e o campo de entrada lado-a-lado
        #self.input1_label = QLabel("Nome:") -> Label da entrada (indica o que o usuário deve inserir)
        #self.input1 = QLineEdit() -> Campo de entrada
        #self.input1.setMaxLength(1) -> Tamanho máximo do campo de entrada (máximo)
        #self.input1.setAlignment(Qt.AlignCenter) -> Alinhar o texto de entrada no centro
        #self.input1_layout.addWidget(self.input1_label) -> Adiciona a label ao layout horizontal
        #self.input1_layout.addWidget(self.input1) -> Adiciona o campo de entrada ao layout horizontal
        #self.main_layout.addLayout(self.input1_layout) -> Adiciona o layout horizontal ao layout principal

        self.input1_layout = QHBoxLayout() 
        self.input1_label = QLabel("Play/Pause:") 
        self.input1 = QLineEdit() 
        self.input1.setMaxLength(1) 
        self.input1.setAlignment(Qt.AlignCenter) 
        self.input1_layout.addWidget(self.input1_label) 
        self.input1_layout.addWidget(self.input1) 
        self.main_layout.addLayout(self.input1_layout) 

        self.input2_layout = QHBoxLayout()
        self.input2_label = QLabel("Avançar Frame:")
        self.input2 = QLineEdit()
        self.input2.setMaxLength(1)
        self.input2.setAlignment(Qt.AlignCenter)
        self.input2_layout.addWidget(self.input2_label)
        self.input2_layout.addWidget(self.input2)
        self.main_layout.addLayout(self.input2_layout)

        self.input3_layout = QHBoxLayout()
        self.input3_label = QLabel("Retroceder Frame:")
        self.input3 = QLineEdit()
        self.input3.setMaxLength(1)
        self.input3.setAlignment(Qt.AlignCenter)
        self.input3_layout.addWidget(self.input3_label)
        self.input3_layout.addWidget(self.input3)
        self.main_layout.addLayout(self.input3_layout)

        self.input4_layout = QHBoxLayout()
        self.input4_label = QLabel("Salvar como 'Indolor':")
        self.input4 = QLineEdit()
        self.input4.setMaxLength(1)
        self.input4.setAlignment(Qt.AlignCenter)
        self.input4_layout.addWidget(self.input4_label)
        self.input4_layout.addWidget(self.input4)
        self.main_layout.addLayout(self.input4_layout)

        self.input5_layout = QHBoxLayout()
        self.input5_label = QLabel("Salvar como 'Pouca dor':")
        self.input5 = QLineEdit()
        self.input5.setMaxLength(1)
        self.input5.setAlignment(Qt.AlignCenter)
        self.input5_layout.addWidget(self.input5_label)
        self.input5_layout.addWidget(self.input5)
        self.main_layout.addLayout(self.input5_layout)

        self.input6_layout = QHBoxLayout()
        self.input6_label = QLabel("Salvar como 'Muita dor':")
        self.input6 = QLineEdit()
        self.input6.setMaxLength(1)
        self.input6.setAlignment(Qt.AlignCenter)
        self.input6_layout.addWidget(self.input6_label)
        self.input6_layout.addWidget(self.input6)
        self.main_layout.addLayout(self.input6_layout)

        self.input7_layout = QHBoxLayout()
        self.input7_label = QLabel("Sair do programa:")
        self.input7 = QLineEdit()
        self.input7.setMaxLength(1)
        self.input7.setAlignment(Qt.AlignCenter)
        self.input7_layout.addWidget(self.input7_label)
        self.input7_layout.addWidget(self.input7)
        self.main_layout.addLayout(self.input7_layout)

        self.input8_layout = QHBoxLayout()
        self.input8_label = QLabel("Abrir arquivo de vídeo:")
        self.input8 = QLineEdit()
        self.input8.setMaxLength(1)
        self.input8.setAlignment(Qt.AlignCenter)
        self.input8_layout.addWidget(self.input8_label)
        self.input8_layout.addWidget(self.input8)
        self.main_layout.addLayout(self.input8_layout)

        # ------------------------------------------------------------------------------------------------

        # Criação dos botões
        self.confirm_button = QPushButton("Confirmar")
        self.close_button = QPushButton("Fechar")

        # Adicionando os botões ao layout
        self.main_layout.addWidget(self.confirm_button)
        self.main_layout.addWidget(self.close_button)

        # Conectando os botões às funções
        self.close_button.clicked.connect(self.close)
        self.confirm_button.clicked.connect(self.confirm_action)

    def confirm_action(self):

        # Obtendo os valores inseridos pelo usuário

        self.playPause = self.input1.text()
        self.nextFrame = self.input2.text()
        self.prevFrame = self.input3.text()
        self.saveIND = self.input4.text()
        self.savePD = self.input5.text()
        self.saveMD = self.input6.text()
        self.exitProg = self.input7.text()
        self.openFile = self.input8.text()

        self.keyValues = [self.playPause, self.nextFrame, self.prevFrame, self.saveIND, self.savePD, self.saveMD, self.exitProg, self.openFile]

        #self.accept()
        self.close()

In [6]:
# Classe responsável pela criação de uma janela específica para os controles do player
class VideoPlayer(QMainWindow):
    
    # Construtor da classe
    def __init__(self):

        super().__init__() # realiza a chamada dos métodos da classe pai (QMainWindow)

        self.setWindowTitle("Video Player com VLC") # Define o título da janela do player
        self.setGeometry(100, 100, 900, 500)  # Define a geometria da janela 

        # Criando um widget central, ele será responsável por aagrupar o conteúdo da janela
        self.central_widget = QWidget(self) # Cria uma instância do QWidget
        self.setCentralWidget(self.central_widget) # Define a instância criada como o central

        # Criando layout horizontal principal do widget central
        self.main_layout = QHBoxLayout() # Cria uma instância do layout
        self.central_widget.setLayout(self.main_layout) # Associa o layout ao widget central

        # Cria um novo widget dedicado para o player de vídeo
        self.video_widget = QVideoWidget() # Cria uma instância do QVideoWidget
        # Associa o widget ao layout central
        # stretch = 1 aumenta o vídeo para ocupar o espaço do vídeo
        self.main_layout.addWidget(self.video_widget, stretch=1) 

        # Criando um widget dedicado para os controles, com uma altura fixa
        self.controls_widget = QWidget() # Cria uma instância do QWidget
        self.controls_widget.setFixedHeight(600)  # Define uma altura fixa para os controles
        self.control_layout = QVBoxLayout(self.controls_widget) # Define um layout vertical para os controles
        self.control_layout.setAlignment(Qt.AlignTop) # Controles alinhados ao topo

        # Associa o widget ao layout central (na orientação horizontal)
        self.main_layout.addWidget(self.controls_widget)

        # Layout horizontal para alguns do botões de controle
        # o qual sera adicionado posteriormente ao control_layout
        # seguindo a ordem de disposição dos botões
        self.hor_layout = QHBoxLayout()
        self.hor_layout2 = QHBoxLayout()
        self.hor_layout3 = QHBoxLayout()

        #-------------------------------------------------------------------------------------------------------------

        # Criação da Janela do Vídeo, sendo ela uma intância 
        # da classe VideoWindow() criada préviamente, além de 
        # associar o VLC à janela criada.

        self.instance = vlc.Instance() # cria uma instância do framework do VLC
        self.media_player = self.instance.media_player_new() # cria um novo player de vídeo a partir da instância do vlc
        self.media_player.set_hwnd(int(self.video_widget.winId())) # Vincula o player do VLC à janela criada

        #-------------------------------------------------------------------------------------------------------------

        # Definição elementos de controle (botões e slider)

        self.slider = QSlider(Qt.Horizontal) # slider
        self.open_button = QPushButton("Abrir Vídeo") # botão para abrir vídeo
        self.next_frame_btn = QPushButton("Avançar frame") # botão para Avançar frame
        self.prev_frame_btn = QPushButton("Retroceder frame") # botão para Retroceder frame
        self.next_selected_frame_btn = QPushButton("Próximo frame salvo") 
        self.prev_selected_frame_btn = QPushButton("Frame salvo anterior") 

        self.play_button = QPushButton("Play/Pause") # botão para dar play ou pause
        self.exit_button = QPushButton("Fechar programa") # botão para sair do programa
        self.save_ind = QPushButton("Salvar frame como 'Indolor'")  # botão responsável por salvar o frame na pasta 'indolor'
        self.save_pd = QPushButton("Salvar frame como 'Pouca dor'") # botão responsável por salvar o frame na pasta 'pouca dor'
        self.save_md = QPushButton("Salvar frame como 'Muita dor'") # botão responsável por salvar o frame na pasta 'muita dor'
        self.change_keys = QPushButton("Escolher novas teclas de atalho") # botão responsável por modificar as teclas de atalho
        self.save_menu = QPushButton("Menu de salvamento do vídeo atual") # botão responsável por abrir o menu de salvamento

        # Definindo slider para o ajuste da velocidade do vídeo

        self.speed_slider = QSlider(Qt.Horizontal) # slider para a velocidade de reprodução do vídeo
        self.speed_label = QLabel("Velocidade: 1x")  # Exibe a velocidade atual como uma label

        # Configurando slider de velocidade
        self.speed_slider.setMinimum(0)  # Definição do menor índice 0 = 0.25x
        self.speed_slider.setMaximum(4)  # Definição do maior índice 4 = 4x
        self.speed_slider.setTickInterval(1)  # Definindo a taxa de incremento de 1 em 1
        self.speed_slider.setTickPosition(QSlider.TicksBelow) # Define a posição do ponteiro do slider
        self.speed_slider.setValue(2)  # Definindo o valor (posição) inicial do slider em 1x

        #-------------------------------------------------------------------------------------------------------------

        # Inserindo os elementos criados na janela de controles
        # observação: A ordem importa para a visualização

        self.control_layout.addWidget(self.play_button)

        # O layout horizontal deve ser inserido em ordem junto com seus widgets
        self.control_layout.addLayout(self.hor_layout)
        self.hor_layout.addWidget(self.next_frame_btn)
        self.hor_layout.addWidget(self.prev_frame_btn)

        self.control_layout.addLayout(self.hor_layout3)
        self.hor_layout3.addWidget(self.next_selected_frame_btn)
        self.hor_layout3.addWidget(self.prev_selected_frame_btn)

        self.time_label = QLabel("00:00 / 00:00") # Instância de um label (formato "00:00 / 00:00")
        self.control_layout.addWidget(self.time_label) # Inserção do label como elemento da janela de controles
        self.control_layout.addWidget(self.slider)

        self.speed_label = QLabel("Velocidade: 1x")  # Exibe a velocidade atual
        self.control_layout.addWidget(self.speed_label)
        self.control_layout.addWidget(self.speed_slider)

        # O layout horizontal deve ser inserido em ordem junto com seus widgets
        self.control_layout.addLayout(self.hor_layout2)
        self.hor_layout2.addWidget(self.save_ind)
        self.hor_layout2.addWidget(self.save_pd)
        self.hor_layout2.addWidget(self.save_md)

        self.control_layout.addWidget(self.open_button)

        

        # Inserção de labels para as teclas de atalho
        self.keys_label = QLabel()
        font = QFont("Arial", 10)  # Nome, tamanho, peso (opcional)
        self.keys_label.setFont(font)
        
        # Define a label como rich text para permitir a inclusão de comandos HTML
        self.keys_label.setTextFormat(Qt.RichText)

        # Valor das teclas de atalho
        self.keys = ["o"," ","d","a","q","j","l","k"]

        self.keys_label.setText(
            "<br>"
            "Teclas de atalho para os controles<br>"
            "<br>"
            f"Abrir um arquivo de vídeo: <b>{self.keys[0]}</b><br>"
            f"Play/Pause: <b>{self.keys[1]}</b><br>"
            f"Avançar frame: <b>{self.keys[2]}</b><br>"
            f"Retroceder frame: <b>{self.keys[3]}</b><br>"
            f"Sair do programa: <b>{self.keys[4]}</b><br>"
            f"Salvar frame como 'indolor': <b>{self.keys[5]}</b><br>"
            f"Salvar frame como 'Pouca dor': <b>{self.keys[7]}</b><br>"
            f"Salvar frame como 'Muita dor': <b>{self.keys[6]}</b><br>"
            "<br>"
        )
        self.control_layout.addWidget(self.keys_label)

        self.control_layout.addWidget(self.save_menu)
        self.control_layout.addWidget(self.change_keys)
        self.control_layout.addWidget(self.exit_button)

        self.frame_label = QLabel("Frame: 0")  # Inicializa com "Frame: 0"
        self.control_layout.addWidget(self.frame_label)  # Adiciona ao layout de controle

        #-------------------------------------------------------------------------------------------------------------

        # Associa as funções de controle aos botões criados 

        self.open_button.clicked.connect(self.open_file) 
        
        self.play_button.clicked.connect(self.toggle_play_pause) 
        
        self.slider.sliderMoved.connect(self.set_position) 
        
        self.exit_button.clicked.connect(self.exit_program) 
        
        self.next_frame_btn.clicked.connect(self.next_frame)
        
        self.prev_frame_btn.clicked.connect(self.prev_frame)

        self.next_selected_frame_btn.clicked.connect(self.next_selected_frame)

        self.prev_selected_frame_btn.clicked.connect(self.prev_selected_frame)
        
        self.speed_slider.valueChanged.connect(self.change_speed)

        self.change_keys.clicked.connect(self.key_mapping)

        # Em Python, uma expressão lambda é uma função anônima de uma única linha.
        # Ela é usada para definir funções curtas sem precisar usar def.
        # A sintaxe base para uma função lambda é "lambda argumentos: expressão"

        # No caso do trecho a baico cada botão precisa chamar a função capture_frame() com um nome de pasta diferente.
        # Contudo o connect() do PyQt exige uma função sem argumentos, por isso, O lambda cria uma função anônima 
        # temporária que será executada somente quando o botão for clicado.

        # Podemos dizer que lambda: self.capture_frame("Indolor") seria o mesmo que:
        # def anonimous():
        #   self.capture_frame("indolor")

        self.save_ind.clicked.connect(lambda: self.capture_frame("Indolor"))
        self.save_pd.clicked.connect(lambda: self.capture_frame("Pouca dor"))
        self.save_md.clicked.connect(lambda: self.capture_frame("Muita dor"))

        self.save_menu.clicked.connect(self.open_save_menu)

        # Timer responsável por atualizar a barra de progresso

        self.timer = QTimer(self) # Cria uma instância do temporizador
        self.timer.setInterval(500) # Define um intervalo de tempo de 500ms
        self.timer.timeout.connect(self.update_slider) # O slider é atualizado a cada 500ms, utilizando a fução update
        
        
        self.timerFrame = QTimer(self)
        self.timer.setInterval(33) 
        self.timerFrame.timeout.connect(self.update_frame_label)  # Atualiza frame junto com o slider

        #-------------------------------------------------------------------------------------------------------------

        # Associando teclas de atalho a cada uma das funções dos botões

        self.open_button_shortcut = QShortcut(QKeySequence(self.keys[0]), self)
        self.open_button_shortcut.activated.connect(self.open_file)

        self.play_button_shortcut = QShortcut(QKeySequence(self.keys[1]), self)
        self.play_button_shortcut.activated.connect(self.toggle_play_pause)

        self.next_frame_shortcut = QShortcut(QKeySequence(self.keys[2]), self)
        self.next_frame_shortcut.activated.connect(self.next_frame)

        self.prev_frame_shortcut = QShortcut(QKeySequence(self.keys[3]), self)
        self.prev_frame_shortcut.activated.connect(self.prev_frame)

        self.exit_shortcut = QShortcut(QKeySequence(self.keys[4]), self)
        self.exit_shortcut.activated.connect(self.exit_program)

        self.save_ind_shortcut = QShortcut(QKeySequence(self.keys[5]), self)
        self.save_ind_shortcut.activated.connect(lambda: self.capture_frame("Indolor"))

        self.save_pd_shortcut = QShortcut(QKeySequence(self.keys[7]), self)
        self.save_pd_shortcut.activated.connect(lambda: self.capture_frame("Pouca dor"))

        self.save_md_shortcut = QShortcut(QKeySequence(self.keys[6]), self)
        self.save_md_shortcut.activated.connect(lambda: self.capture_frame("Muita dor"))

        # Variável de nome do arquivo de vídeo

        self.videoName = ""

    #-------------------------------------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------------------
    
    
    def update_frame_label(self):

        #if self.media_player.is_playing():

        current_time = self.media_player.get_time()  # Tempo atual em ms
        fps = self.media_player.get_fps()  # FPS do vídeo          
        frame_number = int((current_time / 1000) * fps)  # Calcula o número do frame

        self.frame_label.setText(f"Frame: {frame_number}")  # Atualiza o QLabel

        #self.frame_label.setText(f"Frame: {current_time}")  # Atualiza o QLabel
        #print(current_time)
    
    

    #-------------------------------------------------------------------------------------------------------------

    def open_save_menu(self):

        menu = SaveMenu(self.videoName)
        menu.exec_() 
    
    #-------------------------------------------------------------------------------------------------------------

    def key_mapping(self):

        map_window = KeyMapper()
        map_window.exec_() 
        self.keys = map_window.keyValues

        # Recriando os atalhos
        self.open_button_shortcut.setKey(QKeySequence(self.keys[0]))
        self.play_button_shortcut.setKey(QKeySequence(self.keys[1]))
        self.next_frame_shortcut.setKey(QKeySequence(self.keys[2]))
        self.prev_frame_shortcut.setKey(QKeySequence(self.keys[3]))
        self.exit_shortcut.setKey(QKeySequence(self.keys[4]))
        self.save_ind_shortcut.setKey(QKeySequence(self.keys[5]))
        self.save_pd_shortcut.setKey(QKeySequence(self.keys[7]))
        self.save_md_shortcut.setKey(QKeySequence(self.keys[6]))

        # Atualizando a exibição das teclas de atalho na label
        self.keys_label.setText(
            "<br>"
            "Teclas de atalho para os controles<br>"
            "<br>"
            f"Abrir um arquivo de vídeo: <b>{self.keys[0]}</b><br>"
            f"Play/Pause: <b>{self.keys[1]}</b><br>"
            f"Avançar frame: <b>{self.keys[2]}</b><br>"
            f"Retroceder frame: <b>{self.keys[3]}</b><br>"
            f"Sair do programa: <b>{self.keys[4]}</b><br>"
            f"Salvar frame como 'indolor': <b>{self.keys[5]}</b><br>"
            f"Salvar frame como 'Pouca dor': <b>{self.keys[7]}</b><br>"
            f"Salvar frame como 'Muita dor': <b>{self.keys[6]}</b><br>"
            "<br>"
        )

    #-------------------------------------------------------------------------------------------------------------

    def capture_frame(self, folder_name):

        #MODELO    
        os.makedirs(folder_name, exist_ok=True) # cria o diretório para armazenar os frames (caso não exista previamente)

        #INTERFACE
        # Pausa o vídeo, caso ele já não esteja pausado 
        if self.media_player.is_playing():

            self.media_player.pause()

        # O VLC não tem uma função pronta para retornar o numero do frame atual, contudo é possível obter este valor
        # por meio do valor de tempo do frame atual e pela taxa de quadros do vídeo (FPS), com isso temos a equação:
        # frame atual = tempo atual (ms)/1000 * taxa de quadros

        #INTERFACE
        current_Time = self.media_player.get_time() # tempo do frame atual
        fps = self.media_player.get_fps() # taxa de quadros do vídeo

        if fps > 0:
            frame_number = int((current_Time/1000) * fps)
        else:
            frame_number = 'unknown'
        
        frame_path = os.path.join(folder_name, f"frame_{frame_number}_{self.videoName}.png") # Organiza o caminho da imagem salva
        folders = ["Indolor","Pouca dor","Muita dor"]

        print("frame path", frame_path)
        

        for dirs in folders:
            
            if not os.path.exists(dirs):
                continue

            for img_file in os.listdir(dirs):

                if img_file in frame_path:
                    print("image file", img_file)
                    os.remove(f"{dirs}\{img_file}")

        #MODELO
        self.media_player.video_take_snapshot(0, frame_path, 0, 0) # salva o snapshot do vídeo

        # Observação:
        # video_take_snapshot(num, file_path, width, height)
        # num -> Índice da janela de vídeo (sempre 0 se houver apenas um vídeo).
        # file_path -> Caminho onde a imagem será salva
        # width -> Largura da imagem. Se 0, mantém o tamanho original do vídeo.
        # height -> Altura da imagem. Se 0, mantém o tamanho original do vídeo.

    #-------------------------------------------------------------------------------------------------------------

    # Altera a velocidade do vídeo com base no slider
    def change_speed(self):

        # self.speed_values = [1.0, 2.0, 4.0, 8.0, 16.0] # Lista de valores de reprodução
        self.speed_values = [0.25, 0.5, 1.0, 2.0, 4.0] # Lista de valores de reprodução

        index = self.speed_slider.value()  # Obtém o índice da velocidade
        speed = self.speed_values[index]  # Obtém a taxa correspondente
        self.media_player.set_rate(speed)  # Define a nova taxa de reprodução
        self.speed_label.setText(f"Velocidade: {'%0.2f'%(speed)}x")  # Atualiza o rótulo

    #-------------------------------------------------------------------------------------------------------------

    # Função responsável por avançar individualmente um frame
    def next_frame(self):

        # Pausa o vídeo, caso ele já não esteja pausado 
        if self.media_player.is_playing():
            self.media_player.pause()

        # Utiliza um comando do próprio vlc para avançar o frame
        self.media_player.next_frame()

        self.update_frame_label()

     # Função responsável por retroceder individualmente um frame
     # OBS: o vlc não possui uma função pronta para retroceder, por isso, é necessário
     # retroceder manualmente retrocedendo o tempo do vídeo referente à um frame

    #-------------------------------------------------------------------------------------------------------------

    def prev_frame(self):

        # Pausa o vídeo, caso ele já não esteja pausado 
        if self.media_player.is_playing():
            self.media_player.pause()
        
        fps = self.media_player.get_fps()  # Obtém FPS do vídeo
        if fps > 0:
            frame_time = int(1000 / fps)  # Tempo de um frame em ms
            current_time = self.media_player.get_time()  # Tempo atual em ms
            new_time = max(0, current_time - frame_time)  # Garante que não vá abaixo de 0
            self.media_player.set_time(new_time)  # Define o novo tempo

            self.update_frame_label()

    #-------------------------------------------------------------------------------------------------------------

    def next_selected_frame(self):

        v = 0
    
    #-------------------------------------------------------------------------------------------------------------
    
    def prev_selected_frame(self):

        v = 0
    
    #-------------------------------------------------------------------------------------------------------------

    def exit_program(self):

        os._exit(00)

    #-------------------------------------------------------------------------------------------------------------

    def open_file(self):

        # QFileDialog.getOpenFileName() abre uma janela de seleção de aquivos, retornando dois valores:
        # 1 - O caminho do arquivo selecionado (exemplo: "C:/Videos/filme.mp4").
        # 2 - Um valor extra que contém o filtro de arquivos aplicado

        # Parâmetros utilizados
        # self - Referência à janela principal
        # "Abrir Vídeo" - Título da janela de seleção de arquivos.
        # "" - Diretório inicial (se vazio, abre no último local acessado).
        # "Arquivos de Vídeo (*.mp4 *.avi *.mkv)" - Filtro para exibir apenas arquivos de vídeo.

        file_name, _ = QFileDialog.getOpenFileName(self, "Abrir Vídeo", "", "Arquivos de Vídeo (*.mp4 *.avi *.mkv)")
        #self.video_copy = file_name
        
        # Valida se existe um arquivo de vídeo selecionado

        if file_name:

            media = self.instance.media_new(file_name) # Cria um objeto de mídia a parir da instância do  VLC e associa o vídeo selecionado
            self.media_player.set_media(media) # O objeto de mídia (media) é atribuído ao player de vídeo criado pela instância do VLC
            self.media_player.play() # O vlc inicia a reprodução do vídeo carregado
            self.timer.start() # Inicia o timer da barra de progresso
            self.timerFrame.start() # Inicia o timer da barra de progresso

            self.videoName = os.path.basename(file_name)
            self.videoName = self.videoName[:-4]
            #print(self.videoName)

    #-------------------------------------------------------------------------------------------------------------

    def toggle_play_pause(self):

        # Valida se o vídeo está reproduzindo
        # Em caso positivo, o vídeo é pausado. Em caso negativo o vídeo é reproduzido

        if self.media_player.is_playing():
            self.media_player.pause()

        else:
            self.media_player.play()

    #-------------------------------------------------------------------------------------------------------------
    
    # Define a posição do vídeo ao arrastar o slider.
    def set_position(self, value):

        if self.media_player is not None:

            # observação: value está vindo diretamente do QSlider sempre que o usuário interage com ele.
            # Quando o usuário move o slider, o Qt emite sliderMoved(int) automaticamente, passando o valor do slider para set_position.
            # O connect() pode se conectar a funções que aceitam os argumentos esperados pelo sinal.
            # No caso do sliderMoved, ele emite um único inteiro (o valor do slider), e a função 
            # set_position(self, value) está esperando exatamente um argumento além de self.

            duration = self.media_player.get_length()  # Obtém a duração total do vídeo
            new_time = int((value / self.slider.maximum()) * duration) # Obtem a duração do vídeo de acordo com a posição do slider
            self.media_player.set_time(new_time) # Define o tempo de vídeo de acordo com o tempo obtido pela posição do slider
    
    #-------------------------------------------------------------------------------------------------------------
    
    # Atualiza a posição do slider com base no progresso do vídeo.
    def update_slider(self):

        if self.media_player is not None:

            duration = self.media_player.get_length()  # Duração total do vídeo em ms
            current_time = self.media_player.get_time()  # Tempo atual do vídeo em ms

            if duration > 0:  # Garante que a duração do vídeo é válida
                
                # Define um novo valor para o slider, levando em consideração o tempo de vídeo e o limite do slider
                slider_value = int((current_time / duration) * self.slider.maximum())
                self.slider.setValue(slider_value)

            # Atualiza o rótulo do tempo na interface
            current_sec = current_time // 1000
            duration_sec = duration // 1000
            self.time_label.setText(f"{current_sec // 60:02}:{current_sec % 60:02} / {duration_sec // 60:02}:{duration_sec % 60:02}")
    
    #-------------------------------------------------------------------------------------------------------------
    
    # Formatação do tempo como (mm:ss)
    def format_time(self, seconds):

        minutes = seconds // 60
        seconds = seconds % 60
        return f"{minutes:02}:{seconds:02}"

<string>:340: SyntaxWarning: invalid escape sequence '\{'
<>:340: SyntaxWarning: invalid escape sequence '\{'
<string>:340: SyntaxWarning: invalid escape sequence '\{'
<>:340: SyntaxWarning: invalid escape sequence '\{'
C:\Users\marci_wawp\AppData\Local\Temp\ipykernel_2880\50014033.py:340: SyntaxWarning: invalid escape sequence '\{'
  os.remove(f"{dirs}\{img_file}")


In [7]:
app = QApplication(sys.argv)
player = VideoPlayer()
player.show()
sys.exit(app.exec())

SystemExit: 0

C:\Users\marci_wawp\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
